In [1]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bruno\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Usando nltk para procesar las categorias, y 

In [2]:

def process_text(text):
    """
    Funcion que realiza tokeinizacion en base a un texto.

    Args:
        text (string): Palabra u oración para aplicar la tokeinizacin.

    Returns:
        str: Serie de strings.
    """
    # Aplico la teokeinizacion
    stop_words = set(stopwords.words('english'))
    ps = PorterStemmer()
    words = word_tokenize(text)
    words = [ps.stem(word.lower()) for word in words if word.isalnum() and word.lower() not in stop_words]
    return ' '.join(words)



def obtener_palabras_similares(palabra, modelo, topn=3):
    try:
        similares = modelo.similar_by_word(palabra, topn=topn)
        return [palabra for palabra, _ in similares]
    except KeyError:
        return []

def categories_nlp():  
    """
    Funcion que a partir de un dataframe con categorias de columna "name" aplica la funcion *process_text*

    Returns:
        TfidfVectorizer: Matriz TF-IDF represetando  la frecuencia de terminos ponderada por importancia.(necesaria para el modelo.)
    """
    
    
    
    #Genero un dataframe que contenga, las categorias y los negocios para yelp y google.

    # Cambiar por la lectura a la BD

    local_categories_google = pd.read_parquet('../../datasets/processed/bd/7_categories_google.parquet.gz')

    # Cambiar por la lectura a la BD
    local_categories_yelp = pd.read_parquet('../../datasets/processed/bd/8_categories_yelp.parquet.gz')

    #Si se lee de la base de datos business_id ya esta como nombre.
    local_categories_google.rename(columns={'gmap_id':'business_id'},inplace=True)
    local_categories = pd.concat([local_categories_google,local_categories_yelp])

    # Cambiar por la lectura a la BD
    categoires = pd.read_parquet('../../datasets/processed/bd/2_categories.parquet.gz')
    local_categories = pd.merge(local_categories,categoires,on='categories_id',how='inner')
    
    #### Se genera el dataframe local_categories.#####
    
    
    
    local_categories['procceced'] = local_categories['name'].apply(process_text)

    # Si hay mas clase ademas de restaur ej: pizza restaur borra restaur, si no deja igual
    local_categories['procceced'] = local_categories['procceced'].apply(lambda x:x.replace('restaur','') if x!= 'restaur' else x)
    local_categories['procceced'] = local_categories['procceced'].astype(str)
    # Crear una matriz TF-IDF para medir la similitud del contenido
    
    from gensim.models import KeyedVectors

    # Ruta al archivo GoogleNews-vectors-negative300.bin
    ruta_modelo = '../../datasets/extras/model/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'

    # Cargar el modelo
    modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True,limit=500000)
    
    
    local_categories['processed'] = local_categories['procceced'].apply(
    lambda text: ' '.join(
        [
            ' '.join(obtener_palabras_similares(palabra.strip(), modelo)) 
            if palabra in text 
            else palabra 
            for palabra in text.split()
        ]
    )
    )   
    
    local_categories = local_categories[['business_id','name','processed']]
    local_categories.to_parquet('./datasets/locales_categories.parquet') # Guardo el dataset util


    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(local_categories['processed'])
    return tfidf_matrix

    #Proceso para normalizar las categorias

## Gnero las categorias para luego procesarlas y guardarlas

In [3]:
#Genero un dataframe que contenga, las categorias y los negocios para yelp y google.

# Cambiar por la lectura a la BD

local_categories_google = pd.read_parquet('../../datasets/processed/bd/7_categories_google.parquet.gz')

# Cambiar por la lectura a la BD
local_categories_yelp = pd.read_parquet('../../datasets/processed/bd/8_categories_yelp.parquet.gz')

#Si se lee de la base de datos business_id ya esta como nombre.
local_categories_google.rename(columns={'gmap_id':'business_id'},inplace=True)
local_categories = pd.concat([local_categories_google,local_categories_yelp])

# Cambiar por la lectura a la BD
categoires = pd.read_parquet('../../datasets/processed/bd/2_categories.parquet.gz')
local_categories = pd.merge(local_categories,categoires,on='categories_id',how='inner')





In [ ]:
    
# Hago el procesamiento de las categorias con NLTK y los exporto en un pkl
categories_procceced = categories_nlp() 
with open('./tfidf_matrix.pkl', 'wb') as file:
        pickle.dump(categories_procceced, file)

In [4]:
#Modelo de recomendacion usando similitudes con vecinos cercanos


# Cargo la matriz generada del procesamiento
with open('./tfidf_matrix.pkl', 'rb') as file:
        tfidf_matrix = pickle.load(file)


#Defino y entreno al modelo.
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
knn_model.fit(tfidf_matrix)

# Guardo el modelo en un pkl
with open('./modelo_knn.pkl', 'wb') as file:
    pickle.dump(knn_model, file)



FILTRO POR DISTANCIA

In [13]:
# Funcion que calcula la distancia entre dos punto en funcion de las coordenadas
def haversine(lat1, lon1, lat2, lon2):
    
    """
    Esta funcion aplica la distancia hervesine para encontrar la distancia entre dos puntos a partir de sus coordenaadas.
    
    Args:
        lat1 (float): Latitud del primer punto.
        lon1 (float): Longitud del primer punto
        lat2 (float: Latitud del segundo punto.
        lon2 (float): Longitud del segundo punto.
        
    Returns:
        float:Distancia en metros entre dos coordeandas.
    """
    
    # Radio de la Tierra en metros
    R = 6371000.0

    # Convierte las coordenadas de grados a radianes
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Diferencia de latitud y longitud
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Fórmula haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Distancia en metros
    distance = R * c

    return distance

In [14]:
#Funcion que a partir de un id de negocio y una lista ids retorna la distancia entre ese negocio y cada uno de los demas
def distance(business_id,business_id_list,rang=None):
    
    """
    Esta funcion calcula a partir de un negocio y una lista de negocios, la distancia entre los puntos.

    Args:
        business_id(str): Id del negocio.
        business_id_list(list): Lista con id de negocios.
        rang(float,optional) : Maxima distancia(metros) sobre la cual se quiere devolver los negocios.

    Returns:
        pd.DataFrame: Data frame de los negocios tomando en cuenta las distnacias
    """
    
    if rang:
        filtro_distance = rang
    else:   
        filtro_distance = 3000000 #FIltro distancia en metros.
    #Genero un dataframe con los restaurantes de google y yelp
    
    # Cambiar por la lectura a la BD, si se lee de ahi business_id ya esta como nombre
    business_google=pd.read_parquet('../../datasets/processed/bd/5_business_google.parquet.gz') 
    business_google.rename(columns={'gmap_id':'business_id'},inplace=True) 
    
    # Cambiar por la lectura a la BD
    business_yelp=pd.read_parquet('../../datasets/processed/bd/6_business_yelp.parquet.gz') 
    
    # si se lee de la base de datos cambiar stars de business_yelp por avg_stars
    business = pd.concat([business_google[['business_id','name','avg_stars','latitude','longitude']],business_yelp[['business_id','name','avg_stars','latitude','longitude']]])
    #Genero las coordenadas del local al que le quiero encontrar recomendaciones.
    lat_origin,long_origin = business[business['business_id']==business_id]['latitude'].iloc[0],business[business['business_id']==business_id]['longitude'].iloc[0]
    #Filtro solo por los restaurantes que pertenecen a las recomendaciones.
    business = business[business['business_id'].isin(business_id_list)]
    #Calculo la distancia de cada restuarante recomendado al inicial
    business['distance'] = business.apply(lambda row: haversine(lat_origin, long_origin, row['latitude'], row['longitude']), axis=1)
    #Aplico el filtro de distancia.
    business = business[business['distance']<filtro_distance]
    return business
    
    

In [49]:
# Función para obtener recomendaciones
def get_recommendations(business_id,rang=None):
    
    """
    Funcion que a partir de un negocio, recomienda otros, en funcion de sus categorias usando el modelo KNN.
    
    Ags:
        business_id(str) : id del negocio al cual se le quieren calcular recomendaciones.
        rang(float,optional) :Rango de distancia para obtener las recomendaciones(metros).

    Returns:
        business_cat(pd.DataFrame):Data Frame con las recomendaciones junto no algunas caracteristicas del negocio.
    """
    
    # Cargo el modelo
    with open('./modelo_knn.pkl', 'rb') as file: 
        knn_model = pickle.load(file)
        
    with open('./tfidf_matrix.pkl', 'rb') as file:
        categories_procceced = pickle.load(file)
    
    ######### categories_procceced podria ser un df importado  con todos los pasos anteriores.#########
    local_categories = pd.read_parquet('./datasets/locales_categories.parquet')

    idx = local_categories[local_categories['business_id'] == business_id].index[0]
   
    
    
    #Genero las recomendaciones.
    _, indices = knn_model.kneighbors(categories_procceced[idx])
    recommendations = local_categories['business_id'].iloc[indices[0][1:]]  # Excluye el propio restaurante
    
    #Calcula las distancias entre las recomendaciones y el local.
    if rang:
        business = distance(business_id,recommendations,rang)
    else:
        business = distance(business_id,recommendations)
    #Uno las caractereisticas de los locales, con las categorias.
    business_cat = pd.merge(local_categories,business,on='business_id')
    business_cat = business_cat.groupby('business_id').agg({
        'latitude':'first',
        'longitude':'first',
        'name_x':list,
        'name_y':'first',
        'distance':'first',
        'avg_stars':'mean'
        
    }).reset_index()
        
    
    return business_cat


## FILTRO POR USUARIOS

In [47]:
#Funcion que recibe un business id userid o categoria y recomienda locales, tambien puede agregarse el rango en metros de distancia.
def recommendation(business_ids=None,user_id=None,category=None,distance=None):
    """
    Esta funcion a partird e un negocio usuario o categoria recomienda otros negocios, teniendo en cuenta la distancia de ser requerida.
    Para esto la funcion toma un negocio, o selecciona una lista de ellos usando user_id, y categorias, y aplica la funcion *get_recommendations*

    Args:
        business_ids (str, optional): Id de un negocio.
        user_id (str, optional): Id de un usuario.
        category (str, optional): Categoria (nombre).
        distance (float, optional): Distancia en metros.

    Returns:
        pd.DataFrame: Data Frame con ñas recomendaciones y otras caracteristicas(analizar el uso de json)
    """
        
    
    if user_id:
        
        # Cambiar por la lectura a la BD
        df_rg = pd.read_parquet('../../datasets/processed/bd/9_reviews_google.parquet.gz',columns=['user_id','gmap_id','sentiment'])
        df_ry = pd.read_parquet('../../datasets/processed/bd/10_reviews_yelp.parquet.gz',columns=['user_id','gmap_id','sentiment'])
        df = pd.concat([df_rg,df_ry])
        business_ids = df[df['user_id']==user_id].iloc[:10]['gmap_id'].tolist()
        if len(business_ids) == 0:
            return 'Usuario no encontrado.'
        
    if category:
        df_categories = pd.read_parquet('./datasets/locales_categories.parquet')
        business_ids = df_categories[df_categories['name'].str.lower().str.contains(category.lower())].sample(10).iloc[:10]['business_id'].tolist()
        if len(business_ids) == 0:
            return 'Categoria no encontrada.'
        
    business_cat = pd.DataFrame()
    for business_id in business_ids:
        business_cat = pd.concat([business_cat,get_recommendations(business_ids,rang=distance)])    
    
    if business_cat.shape[0] == 0:
        return 'Restaurante no encontrado.'
    return business_cat

In [50]:
recommendation(business_ids='0x88db4147b1d9e6f3:0x943dbd10a92ba1b1')

,business_id,latitude,longitude,name_x,name_y,distance,avg_stars
0,DcRWas1_U96tePNdwl5vSQ,39.744405,-75.224682,"[Food, Restaurants, Delis, Salad, Barbeque, St...",The Mullica Hill Amish Market,1.584081e+06,4.45
1,H6FkrNQSCZAcYQPn0emM2g,39.843540,-75.016619,"[Food, Restaurants, Event Planning & Services,...",Tropical Hut,1.601664e+06,4.10
2,HoxZwJdokeV0g_1n5L5OLw,39.721306,-75.116827,"[Food, Restaurants, Chinese, Buffets]",Peking Buffet,1.585707e+06,3.31
3,R84qWSVghknKWKXPbKwYLA,39.825133,-75.159509,"[Food, Restaurants, Burgers, Fast Food, Coffee...",McDonalds,1.594574e+06,1.55
4,S57Jpd0ZAv7CEf8EIJopNQ,39.828419,-74.999730,"[Seafood, Food, Restaurants, Gluten-Free, Asia...",Nimit Indian Kitchen,1.600775e+06,3.94
...,...,...,...,...,...,...,...
4,S57Jpd0ZAv7CEf8EIJopNQ,39.828419,-74.999730,"[Seafood, Food, Restaurants, Gluten-Free, Asia...",Nimit Indian Kitchen,1.600775e+06,3.94
5,gGjjMoU0zGstBWI0pz60gQ,39.905725,-74.964926,"[Pizza, Food, Restaurants, Italian, Ethnic Foo...",Romanzas Pizza Italian Restaurant,1.609808e+06,3.73
6,goyuP6nqBsW1kuOEk7i4Sg,39.915693,-75.020945,"[Food, Restaurants, Delis, Italian, Sandwiches...",Croces Pasta,1.608740e+06,4.52
7,ki5kQi14Pn7S--GJVvP8aA,39.989292,-75.008944,"[Food, Restaurants, Delis, Fast Food, Sandwiches]",Jersey Mikes Subs,1.616568e+06,3.19


# AHORA INTENTAMOS CON WORDVEC

In [18]:
local_categories = pd.read_parquet('./datasets/locales_categories.parquet') 

In [ ]:
local_categories

,business_id,categories_id,business_id_int,name,procceced,weighted_categories
0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,0,0,Restaurant,restaur,0.01 restaur
1,0x8890b9241e704667:0x3a1e565c17c00993,0,1,Restaurant,restaur,0.01 restaur
2,0x88e635378f43352f:0xa1b53c63436fa428,0,2,Restaurant,restaur,0.01 restaur
3,0x88d9b86c23bb04eb:0x2a82da546b7feb1b,0,3,Restaurant,restaur,0.01 restaur
4,0x88d9ba5d65937567:0xbc27649cf513cc89,0,4,Restaurant,restaur,0.01 restaur
...,...,...,...,...,...,...
203650,vV57YWbrHqm1iylWmIdwVA,1653,66978,Botanical Gardens,botan garden,1.0 botan 1.0 garden
203651,y7P06O7ypUgdU5fgV4s87Q,1654,58724,Newspapers & Magazines,newspap magazin,1.0 newspap 1.0 magazin
203652,jchZL8NJP9YBwgFmrSdpfQ,1655,67246,Parking,park,1.0 park
203653,9295K2aKOltQSSTpm_7DBQ,1656,67285,Georgian,georgian,1.0 georgian


In [ ]:
from gensim.models import KeyedVectors

# Ruta al archivo GoogleNews-vectors-negative300.bin
ruta_modelo = '../../datasets/extras/model/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'

# Cargar el modelo
modelo = KeyedVectors.load_word2vec_format(ruta_modelo, binary=True,limit=500000)



In [ ]:
def obtener_palabras_similares(palabra, modelo, topn=3):
    try:
        similares = modelo.similar_by_word(palabra, topn=topn)
        return [palabra for palabra, _ in similares]
    except KeyError:
        return []

In [ ]:
print(obtener_palabras_similares('restaurant',modelo))

['eatery', 'Restaurant', 'restuarant']


In [ ]:
local_categories.sample(3)

,business_id,categories_id,business_id_int,name,procceced,weighted_categories,processed
139072,0x809107130f8bd273:0xf8c2a452777d83b,541,13649,Community center,commun center,1.0 commun 1.0 center,centers facility hub
90859,0x88c3138458455555:0xe004d8781d4f55a5,47,36598,Italian restaurant,italian,1.0 italian,restaur
15199,0x80dbf48d10967113:0xff8971dc5197208,0,15199,Restaurant,restaur,0.01 restaur,restaur


In [ ]:
local_categories['processed'] = local_categories['procceced'].apply(
    lambda text: ' '.join(
        [
            ' '.join(obtener_palabras_similares(palabra.strip(), modelo)) 
            if palabra in text 
            else palabra 
            for palabra in text.split()
        ]
    )
)

In [ ]:
local_categories.loc[local_categories['processed'] == '', 'processed'] = 'restaur'

In [ ]:
local_categories.sample(5)

,business_id,categories_id,business_id_int,name,procceced,weighted_categories,processed
106106,0x80dcae429c639ce1:0xa04bc862f4da241,68,8493,Salad shop,salad shop,1.0 salad 1.0 shop,salads veggies soup shops store Shop
104007,0x88d927441c04d3cb:0x1aeba730985c786,61,50076,Barbecue restaurant,barbecu,1.0 barbecu,restaur
165496,4D7QDfPOsSPfKxBi3gs_qQ,1159,64782,Italian,italian,1.0 italian,restaur
178818,tccw4OnMjHCoQ_Y0A145cw,1174,67557,Sandwiches,sandwich,1.0 sandwich,sandwiches burger burgers
24213,0x88c3412a3ee2e799:0xa39ec1bdfb2ffe8a,1,24198,Buffet restaurant,buffet,1.0 buffet,appetizers hors_d'_oeuvres brunch


In [ ]:



tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(local_categories['processed'])



#Defino y entreno al modelo.
knn_model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10)
knn_model.fit(tfidf_matrix)

# Guardo el modelo en un pkl
with open('./model/modelo_knn.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

In [ ]:
#Genero las recomendaciones.
_, indices = knn_model.kneighbors(tfidf_matrix[106106])
recommendations = local_categories['business_id'].iloc[indices[0][1:]]  # Excluye el propio restaurante

In [ ]:
print(local_categories.iloc[106106])

In [ ]:
print(recommendations)

105915    0x88d905f9b0dccd0d:0x189d7a76056de69a
105919     0x88e8a47bf0404f75:0x174e53d42a97e38
105918    0x88ef3022384aa383:0xba8301cfe7cf949a
105917    0x88d9b7ec20abf63f:0xcb586271c48039e9
105921    0x88d9a8594d5fe633:0x8e805fcab26ef0e2
105912    0x88e5b510280178e5:0xed333707709151e6
105913    0x88dc2abc8fe6630b:0x319527453eeda1ca
105922    0x88c33922730d1d0b:0xf3a0d96f1af92e3b
105914    0x88d8e1c9c8080dc5:0xf4f5e1f051b47c13
Name: business_id, dtype: object


In [ ]:
local_categories[local_categories['business_id']=='0x88d905f9b0dccd0d:0x189d7a76056de69a']

,business_id,categories_id,business_id_int,name,procceced,weighted_categories,processed
57896,0x88d905f9b0dccd0d:0x189d7a76056de69a,19,36537,Pizza restaurant,pizza,1.0 pizza,pizzas Pizza sandwich
90797,0x88d905f9b0dccd0d:0x189d7a76056de69a,47,36537,Italian restaurant,italian,1.0 italian,restaur
105625,0x88d905f9b0dccd0d:0x189d7a76056de69a,67,36537,Pasta shop,pasta shop,1.0 pasta 1.0 shop,spaghetti salad noodles shops store Shop
105915,0x88d905f9b0dccd0d:0x189d7a76056de69a,68,36537,Salad shop,salad shop,1.0 salad 1.0 shop,salads veggies soup shops store Shop
